### In this notebook translates the NewCensusBlocks.geojson generated before with the affected areas marked by the 2 walkings distances and translate them to the census track level.
### Census tracks are less granular and will help us to speed up computations

In [1]:
import zipfile 
import pysal as ps
%matplotlib inline
import numpy as np
import ast
import json
import urllib
import urllib2
from cStringIO import StringIO
import pandas as pd
import requests
import os
import geopandas as gpd
import matplotlib.pylab as pl
from fiona.crs import from_epsg
import matplotlib.pylab as pl
from matplotlib import colors
import shapely
import seaborn as sns
import gzip
import mapreduce as mr
import csv

In [2]:
# Load the file that we generated for the Census Blocks
blocks = gpd.GeoDataFrame.from_file('Data/NewCensusBlocks.geojson')
blocks[blocks.Within30min==True].head()

,OnRangeToL,Within15min,Within30min,bctcb_2010,boro_code,boro_name,cb_2010,ct_2010,geometry,id,shape_area,shape_leng
3098,0,0,1,10002021000,1,Manhattan,1000,000202,(POLYGON ((-73.9799619880113 40.71397269930172...,3098,57115.9140286,1187.88386045
3101,0,0,1,10012001000,1,Manhattan,1000,001200,(POLYGON ((-73.98247747854293 40.7182905071724...,3101,119406.482234,1428.26565607
3102,0,0,1,10014011000,1,Manhattan,1000,001401,(POLYGON ((-73.9883662631773 40.71644570203691...,3102,326876.540881,2343.21689659
3104,0,0,1,10016001000,1,Manhattan,1000,001600,(POLYGON ((-73.99309432171637 40.7179224787339...,3104,192194.344295,2216.0191299
3105,0,0,1,10018001000,1,Manhattan,1000,001800,(POLYGON ((-73.98787935797355 40.7174089725307...,3105,88287.4881612,1218.95689976


In [3]:
blocks.crs

{}

In [15]:
#Get the Census track code
def getrack_blockname(x):
    return x[1:]
blocks['track_block'] = blocks.bctcb_2010.map(getrack_blockname)
def geboro_track(x):
    return x[:-4]
blocks['boro_track'] = blocks.bctcb_2010.map(geboro_track)

In [16]:
blocks.head()

,OnRangeToL,Within15min,Within30min,bctcb_2010,boro_code,boro_name,cb_2010,ct_2010,geometry,id,shape_area,shape_leng,track,track_block,boro_track
0,0,0,0,50009001000,5,Staten Island,1000,000900,(POLYGON ((-74.07920577013245 40.6434307837456...,0,244589.603884,2508.94811457,0009001000,0009001000,5000900
1,0,0,0,50020011000,5,Staten Island,1000,002001,(POLYGON ((-74.07061992438017 40.6108550670541...,1,111006.286826,1345.88642172,0020011000,0020011000,5002001
2,0,0,0,50027001000,5,Staten Island,1000,002700,(POLYGON ((-74.07524403910642 40.6260063245271...,2,150406.766022,1703.38117241,0027001000,0027001000,5002700
3,0,0,0,50040001000,5,Staten Island,1000,004000,(POLYGON ((-74.08708799689826 40.6111688312392...,3,141296.561149,1511.17374266,0040001000,0040001000,5004000
4,0,0,0,50064001000,5,Staten Island,1000,006400,(POLYGON ((-74.07643744511992 40.6010016065558...,4,200784.983607,1978.24385199,0064001000,0064001000,5006400


### Load the Census tracks shapefiles

In [9]:
url='https://data.cityofnewyork.us/api/geospatial/fxpq-c8ku?method=export&format=GeoJSON'
filebuffer = urllib.urlretrieve(url, filename='CensusTracks.geojson')
fname = 'CensusTracks.geojson'
tracks = gpd.GeoDataFrame.from_file(fname)
print tracks.crs 

{}


In [17]:
tracks['Within30min']=False
tracks['Within15min']=False
tracks.head()

,boro_code,boro_ct_2010,boro_name,cdeligibil,ct_2010,ctlabel,geometry,ntacode,ntaname,puma,shape_area,shape_leng,Within30min,Within15min
0,5,5000900,Staten Island,I,000900,9,(POLYGON ((-74.07920577013245 40.6434307837456...,SI22,West New Brighton-New Brighton-St. George,3903,2497009.69813,7729.01679376,False,False
1,5,5007400,Staten Island,I,007400,74,(POLYGON ((-74.05974734759452 40.5938486115672...,SI14,Grasmere-Arrochar-Ft. Wadsworth,3902,5788237.79601,9902.94847281,False,False
2,1,1009800,Manhattan,I,009800,98,(POLYGON ((-73.96432543478758 40.7563815309909...,MN19,Turtle Bay-East Midtown,3808,1906016.35002,5534.19981063,False,False
3,1,1010000,Manhattan,I,010000,100,(POLYGON ((-73.96802436915851 40.7595781400528...,MN19,Turtle Bay-East Midtown,3808,1860938.37721,5692.16873705,False,False
4,1,1010200,Manhattan,I,010200,102,(POLYGON ((-73.97124277307127 40.7609364184790...,MN17,Midtown-Midtown South,3807,1860992.70485,5687.80243866,False,False


In [18]:
#create the columns in the census tracks data.
def at30walkingdistance(track):
    if track in list(blocks.boro_track[blocks.Within30min==True]):
        return True
    return False
def at15walkingdistance(track):
    if track in list(blocks.boro_track[blocks.Within15min==True]):
        return True
    return False
tracks['Within30min']=tracks.boro_ct_2010.map(at30walkingdistance)
tracks['Within15min']=tracks.boro_ct_2010.map(at15walkingdistance)

In [20]:
tracks[tracks['Within30min']==True]

,boro_code,boro_ct_2010,boro_name,cdeligibil,ct_2010,ctlabel,geometry,ntacode,ntaname,puma,shape_area,shape_leng,Within30min,Within15min
6,1,1011300,Manhattan,I,011300,113,(POLYGON ((-73.98411754823195 40.7548420534646...,MN17,Midtown-Midtown South,3807,1890907.29102,5699.8606412,True,False
100,3,3026300,Brooklyn,E,026300,263,(POLYGON ((-73.9420768492443 40.68869720248089...,BK75,Bedford,4003,1287405.97132,4799.55684944,True,False
112,3,3027900,Brooklyn,E,027900,279,(POLYGON ((-73.93617169991664 40.6886267476856...,BK35,Stuyvesant Heights,4003,1743087.9778,5389.74582137,True,False
114,3,3028501,Brooklyn,I,028501,285.01,(POLYGON ((-73.93856854782675 40.6988481355494...,BK78,Bushwick South,4002,895739.69679,4167.68973824,True,True
116,3,3029100,Brooklyn,E,029100,291,(POLYGON ((-73.93032510585428 40.6892972646220...,BK35,Stuyvesant Heights,4003,1772059.65213,5439.93915313,True,False
117,3,3029300,Brooklyn,E,029300,293,(POLYGON ((-73.93017996947596 40.6885651406574...,BK35,Stuyvesant Heights,4003,1770060.57626,5437.78521721,True,False
118,3,3029700,Brooklyn,I,029700,297,(POLYGON ((-73.9290088401841 40.68270290548576...,BK61,Crown Heights North,4006,2212768.01658,5858.60267244,True,False
122,3,3030900,Brooklyn,E,030900,309,(POLYGON ((-73.9303176269034 40.67741303826242...,BK61,Crown Heights North,4006,1587377.38042,5470.58144347,True,False
133,3,3036300,Brooklyn,E,036300,363,(POLYGON ((-73.91400482503951 40.6732394879431...,BK79,Ocean Hill,4007,2785736.31011,8445.46688322,True,True
134,3,3036502,Brooklyn,E,036502,365.02,(POLYGON ((-73.90267831840295 40.6757897287538...,BK79,Ocean Hill,4007,1252061.56461,6132.19606711,True,True


In [25]:
with open('Data/CensusBlocks.geojson', 'w') as f:
    f.write(blocks.to_json())

with open('Data/CensusTracks.geojson', 'w') as f:
    f.write(tracks.to_json())

### Save the geojson files with the new columns included for future use